In [4]:
import pandas as pd

In [10]:
olymp = pd.read_excel("olympics-data.xlsx") # to read an excel file 

olymp.head() # to get the first 5 values of the data

,athlete_id,name,born_date,born_city,born_region,born_country,NOC,height_cm,weight_kg,died_date
0,1,Jean-François Blanchy,1886-12-12,Bordeaux,Gironde,FRA,France,NaN,NaN,1960-10-02
1,2,Arnaud Boetsch,1969-04-01,Meulan,Yvelines,FRA,France,183.0,76.0,NaN
2,3,Jean Borotra,1898-08-13,Biarritz,Pyrénées-Atlantiques,FRA,France,183.0,76.0,1994-07-17
3,4,Jacques Brugnon,1895-05-11,Paris VIIIe,Paris,FRA,France,168.0,64.0,1978-03-20
4,5,Albert Canet,1878-04-17,Wandsworth,England,GBR,France,NaN,NaN,1930-07-25


# Accessing Data with Pandas 

In [ ]:
olymp.tail() # to get the last 5 values of the data

In [ ]:
# to access specific values in our data: classic index slicing
olymp.loc[0:3] # will get us the file until the 3rd piece of data

olymp.loc[0:2, ["name", "NOC"]] # this format allows us to select specific rows and columns. eg: this piece of code will only grab
# the name and the NOC of the players from index 0 to 3


In [ ]:
# the iloc allows --> .iloc[row_selection, column_selection] 
# a key difference to note between loc and iloc is that, with .iloc, the upper index is not inclusive

olymp.iloc[0:3, [0, 1, 2, 4, 5]] # NOTICE: how to the 3rd column (born_city) is missing 

In [ ]:
# Notice how the indexing, which originally started from 0, has now changed to the the exact data of athlete_id.
# that makes each row uniquely identified by an athlete’s ID, 
# which is helpful for fast lookups, data merging, and removing duplicates based on athletes.

olymp.index = olymp['athlete_id']
olymp.loc[0] #since index 0 has now been removed, even .loc would not recognize it and show it be a "KeyError"
olymp.loc[1] # this has replaced loc and the index 1 now shows the data of the athlete "Jean-François Blanchy"

In [ ]:
olymp.sample(n=6) # .sample(n=k) randomly select rows (or columns), you can see that everytime you run the code, the data changes

olymp.sample(frac=0.2) # create a test sample: .sample(frac=k) will get k% of the data, 
                      # eg: this code will show some random 20% of the whole data

olymp.sample(frac=1) # shuffle the dataset

olymp.sample(n=2, random_state=1) # keep the same set of random result each time in each random state
# this is called reproducibility. ensures that you, and others, get the same result everytime
# your random splits stay consistent during model training


In [ ]:
olymp.head()

In [ ]:
olymp.sort_values("born_date") # this will sort out values based on the date of birth in an increasing order

olymp.sort_values("born_date", ascending = False) # this will sort out values based on the date of birth in a decreasing order

olymp.sort_values(["height_cm", "born_date"]) # this will first sort according to the height, and then through their born date

# Filtering Data

In [ ]:
# to grab a certain type of data. With .loc, we only see a certain number of data. 

olymp.loc[olymp['height_cm'] > 200] # to grab the atheletes whose height is more than 200 cm. 

olymp.loc[olymp['height_cm'] > 200, ['name', 'height_cm']] # will only show the specific columns mentioned


In [ ]:
# Without .loc, we get to see the whole dataset where the height > 215

olymp[olymp['height_cm'] > 215]

olymp[(olymp['height_cm'] > 215) & (olymp['born_country'] == 'USA')] # shows two conditions combined with the AND operator. 
                                             # only the athletes of height > 215 and who were born in the USA will show up

In [ ]:
# .contains checks whether the data contains the listed input 

olymp[olymp['name'].str.contains("Keith")] # notice how "keith" with a lower case k will fetch us no results. (case-sensitive)

olymp[olymp['name'].str.contains("Keith|patrick", case = False)] # checks if the data contains both keith and patrick (using regex)
# case = False renders the case sensitivity to be False. Even though, we wrote "patrick" , the data still showed "Patrick"

olymp[olymp['born_country'].isin(["USA", "GBR"]) & (olymp['name'].str.startswith('John'))] 
# athletes whose name is John and are from either the USA or the Great Britain

# an easier way to filter data:
olymp.query('born_country == "USA"')


# Adding/Removing a Column

In [ ]:
# let's add a column where if the athlete is French, the new column will say so

import numpy as np 

olymp['French?'] = np.where(olymp['born_country'] == "FRA", 'French', 'Not French')
# we created a column called "French?" where if NOC == "France", then the new column will say "French", otherwise it will say "Not French"

olymp.head(77)


In [ ]:
# lets create a new column where we have the product of each athlete's height and weight (why not?)

olymp["prod"] = olymp['height_cm'] * olymp['weight_kg']
olymp.head()

# lets remove the columns that we just created cause its not like we actually need it
rem = ['prod', 'French?']

olymp.drop(rem, axis =1, inplace = True)
olymp.head()

# the examples above were to show that you can use mathetical operations in the creation of new columns

In [ ]:
# to rename a column 

olymp.rename(columns={'NOC': 'Country Represented'}, inplace=True)

olymp.head()

In [ ]:
# to add two columns that display the first name and last name seperately 

olymp['first name'] = olymp['name'].str.split(" ").str[0]

olymp['last name'] = olymp['name'].str.split(" ").str[1]

olymp.head()

In [ ]:
# when you do .info(), you can see how the datatype is an object for born date.  We should make it into a datetime Dtype

olymp['born_date'] = pd.to_datetime(olymp['born_date'])

olymp.info() # now we can see that the Dtype has been changed from object to datetime64[ns]
olymp.head()

In [ ]:
# .dt works like .str but is for datetime format

olymp['born_year'] = olymp['born_date'].dt.year

olymp[['name','born_year']] # will only get the name and the year, the rest of the columns will not be shown


In [ ]:
# using lambda to create a new category

olymp['height_category'] = olymp['height_cm'].apply(lambda x: 'Short' if x < 165 else ('Average' if x < 185 else 'Tall'))

olymp.head(100)

In [ ]:
def categorize(row):
    if row['height_cm'] < 175 and row['weight_kg'] < 70:
        return "LightWeight"

    elif row['height_cm'] < 185 and row['weight_kg'] <= 80:
        return "MiddleWeight"

    else:
        return "HeavyWeight"

olymp['Category'] = olymp.apply(categorize, axis =1)

olymp.head()


# Merging and Concatenating Data

In [ ]:
# using the logical OR

new = olymp[(olymp['born_region'] == 'New Hampshire') | (olymp['born_city'] == 'San Francisco')] 
# will get the data of athletes who are either from New Hampshire or San Francisco

# using the AND operator
old = olymp[(olymp['born_region'] == 'New Hampshire') & (olymp['born_city'] == 'San Francisco')] 
# since there are no athletes who are from New Hampshire and San Francisco at the same time, there will be no data on this whatsoever

new.head()
old.head()

In [ ]:
# how can we get the data of atheletes who are from a specific place

usa = olymp[olymp['born_country'] == 'USA'].copy()
gbr = olymp[olymp['born_country'] == "GBR"].copy()

usa.head() # will show everyone who is from the USA
gbr.head() # will show everyone who is from the Great Britain

# how can we merge these two data together so that the file shows us athletes from both, the USA and the GBR, at the same time.

new_df = pd.concat([usa, gbr])  # using .concat

new_df.head() 
new_df.tail() 

In [ ]:
# Now, we have some new raw data that I want to merge with the olympics dataframe

results = pd.read_csv('Olymps_results.csv')


# lets combine the two data together

# since athelete_id is the only thing both the data had in common, we use that. However, both our index is also called "athlete_id"
# hence, this will show an error 

# We now have to specify for the files to reset_index 
if 'athlete_id' in olymp.index.names:
    olymp = olymp.reset_index(drop = True)
if 'athlete_id' in results.index.names:
    result = result.reset_index(drop = True)

combined = pd.merge(olymp, results, on = 'athlete_id', how = 'left')

combined.head(20)


# Handling Null Values 

In [ ]:
# let us change the word "NaN" to just a dash "-"

combined = combined.fillna("-") # every data that said NaN beforehand, now says -, much easier to read and comprehend

# when working with numbers, we can also use .interpolate that will fill NaN with values based on patterns beside it 

# to drop the whole row that has Nan values, we can use 

results.dropna(inplace = True) # notice how the index now starts with and skips some number slots along, we have removed all the data with NaN values
# be careful with this one since it drops the entire row, inplace = True makes it permanent


results.head(15)

combined.head(15) # notice how the changed we implemented above only exist in results dataframe. In "combined", we replaced Nan values with "-"




In [ ]:
# to get the rows that have Nan Values only 
results[results['team'].isna()] # this will result in nothing since we do not have any data where the team column has a Nan

# to get the rows that have no Nan Values whatsoever

results[results['team'].notna()] # since there are no nans in our data file, we get our all the rows here



# Aggregating Data

In [ ]:

olymp['born_city'].value_counts() # counts the amount of athletes from each city

olymp[olymp['born_country'] == 'USA']['born_region'].value_counts().tail(25)  # counts the amount of athletes from each city


In [ ]:
coffee = pd.read_csv('coffee.csv') # for the next examples, let us use another set of data 


coffee['Price'] = np.where(coffee['Coffee Type'] == 'Espresso', 5.99, 3.99)

coffee['Revenue'] = coffee['Units Sold'] * coffee['Price'] 
# here, I created another column that represents the Revenue

coffee.groupby(['Coffee Type'])['Units Sold'].mean()

coffee.groupby(['Coffee Type']).agg({'Units Sold' : 'sum', 'Price' : 'mean'})
 # groups the data by coffee type and shows the sum of the units sold and the average price of each type: using aggregation 

# using named aggregation 
coffee.groupby(['Coffee Type']).agg(
    sum_sold = ('Units Sold', 'sum'),
    average_price = ('Price', 'mean')
)  
    

In [ ]:
# using pivot makes it easier to grab information 

pivot = coffee.pivot(columns='Coffee Type', index = 'Day', values = 'Revenue')

pivot.loc['Monday', 'Latte'] # easy to grab monday's latte count or any other specific value

pivot.sum() # shows the total revenue (goes through the columns). It can also be written as pivot.sum(axis=0), which is the default setting

pivot.sum(axis=1) # goes through rows, so shows the sum of each day 

In [ ]:
# lets aggregate data on our olympics file
olymp['born_date'] = pd.to_datetime(olymp['born_date'], errors='coerce')
# errors = 'coerce' basically tells pandas "if you find something that cannot be turned into a date, do not crash and just replace it
# with a special missing value "NaT".   Now, even if we remove errors ='coerce', the code wont crash since the system has already replaced 
# spaces with Nat

olymp.groupby(olymp['born_date'].dt.year)['name'].count()
# groups by born_date and shows how many names are in those years

olymp.groupby(olymp['born_date'].dt.year)['name'].count().reset_index().sort_values('name')
# groups the data by born_date and shows how many names in those years, turns the results back from series into a dataframe, then sorts them
# in an ascending order

olymp.groupby(olymp['born_date'].dt.year)['name'].count().reset_index().sort_values('name', ascending = False)
# does the same thing as the code above but in an ascending order





reset_index() turns that index back into a regular column, which gives you a DataFrame with normal columns instead of grouped index labels.

This makes it easier to:

Sort by multiple columns
Rename columns
Save or export the results
Merge with other DataFrames

In [ ]:
olymp['born_date'] = pd.to_datetime(olymp['born_date'], errors='coerce')

olymp['month_born'] = olymp['born_date'].dt.month
olymp['year_born'] = olymp['born_date'].dt.year

olymp.groupby([olymp['year_born'], olymp['month_born']])['name'].count().reset_index().sort_values('name')
# This code groups the data by both birth year and birth month, counts how many names fall into each (year, month) pair,
# converts the result to a DataFrame, and then sorts the rows by the count of names in ascending order.





# Advanced Functionality

In [ ]:
coffee["yesterday's Revenue"] = coffee['Revenue'].shift(2)
# .shift() moves values down by 1 row. shift(2) is for 2 rows which makes more sense here given that we have two days for each day

# we can also calculate the percentage change in today's revenue and yesterday's revenue

coffee['pct_change'] = coffee['Revenue']/coffee["yesterday's Revenue"] *100 
#shows the percentage change between 

coffee 

In [ ]:
#  combined['height_rank'] = combined['height_cm'].rank()

#  combined.sort_values(['height_rank'], ascending = False) 
# here, using combined values will not work given that our combined data has a mix of string and float values. Remember, that in our
# Combined dataframe, we have replaced all NaN values with "-"

olymp['height_rank'] = olymp['height_cm'].rank() # ranks the data according to the height of each athlete

olymp.sort_values(['height_rank'], ascending = False), # shows the tallest person first


In [ ]:
# using .cumsum():  cummulative summation 

coffee['cummulative_rev'] = coffee['Revenue'].cumsum()  # adds a new column that shows the cummulative revenue 

coffee['3_day_revenue'] = coffee['Units Sold'].rolling(3).sum() 
# .rolling() is used to view over your data so you can use .sum(), .mean(), .max(), .min()
# it returns a rolling object, so we have to follow it with an aggretion function

# if we want to check the 3 day revenue only for Lattes, we can:
latte = coffee[coffee['Coffee Type'] == 'Latte'].copy()

latte['latte_3day'] = latte['Units Sold'].rolling(2).sum()
# the number inside of rolling represents "window" = number of observations 
# can also be written as rolling(window = 2)

latte # we can now see a column with a 

In [ ]:
# Bonus: to check your pandas version 

pd.__version__  

In [ ]:

def summarize_max(frame, name = 'None', val = 'None', description = 'Maximum'):
    if isinstance(frame, pd.Series):     # if it is a series, 
        max_ind = frame.idxmax()         # gets the index with the maximum value
        max_val = frame[max_ind]         # uses that maximum index to get that maximum value
        print(f'{description}: {max_ind} with a value of {max_val}')
    else:
        # For DataFrame 
        max_row = frame[val].idxmax()          # gets the row index of the maximum value
        max_label = frame.loc[max_row, name]   # gets the label for that row
        max_val = frame.loc[max_row, val]      # gets the numerical value for that row
        print(f'{description}: {max_label} with a value of {max_val}') 
    return max_ind, max_val

'''
frame: either a pandas Series (like event_counts) or DataFrame (like a grouped DataFrame).

name: if it’s a DataFrame, this is the column that holds the label (like sport/event name).

value: the column with the numeric values you want to find the max of.

description: a text string to describe what we are reporting in the sentence.
'''


In [ ]:
combined['Age_of_Competition'].max()

combined.loc[combined['Age_of_Competition'].idxmax(), ['name', 'birth_year', 'year', 'Age_of_Competition']]